## Part D: Increase the number of hidden layers

In [1]:
import pandas as pd
import numpy as np

#Downloading and Reading the DATA
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Split data into predictors and target

In [2]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

##### Normalize the data

In [3]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [4]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

#### Build the Neural Network

In [5]:
#import relevant libraries
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten, Conv2D, MaxPooling2D

In [6]:
# define regression model
# added 3 hidden layers
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [8]:
# build the model
model = regression_model()

# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=2)

Train on 721 samples
Epoch 1/50
721/721 - 13s - loss: 1552.9083
Epoch 2/50
721/721 - 14s - loss: 1498.9443
Epoch 3/50
721/721 - 15s - loss: 1425.7244
Epoch 4/50
721/721 - 14s - loss: 1326.2000
Epoch 5/50
721/721 - 14s - loss: 1192.3250
Epoch 6/50
721/721 - 14s - loss: 1018.4225
Epoch 7/50
721/721 - 14s - loss: 815.4000
Epoch 8/50
721/721 - 14s - loss: 610.0393
Epoch 9/50
721/721 - 12s - loss: 439.9346
Epoch 10/50
721/721 - 12s - loss: 336.8723
Epoch 11/50
721/721 - 11s - loss: 284.4992
Epoch 12/50
721/721 - 13s - loss: 259.0952
Epoch 13/50
721/721 - 12s - loss: 241.1304
Epoch 14/50
721/721 - 14s - loss: 226.8318
Epoch 15/50
721/721 - 13s - loss: 216.3291
Epoch 16/50
721/721 - 14s - loss: 207.4814
Epoch 17/50
721/721 - 14s - loss: 202.4095
Epoch 18/50
721/721 - 14s - loss: 195.2529
Epoch 19/50
721/721 - 13s - loss: 191.3797
Epoch 20/50
721/721 - 13s - loss: 186.5185
Epoch 21/50
721/721 - 13s - loss: 182.4451
Epoch 22/50
721/721 - 13s - loss: 179.0893
Epoch 23/50
721/721 - 13s - loss: 17

#### Evaluate the model

In [9]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 2s 7ms/sample - loss: 134.4501


134.45007126693972

In [10]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

134.45007474430523 0.0


In [ ]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

This takes too long to compile. Please review the code seperately. I don't have the right system to run it